<a href="https://colab.research.google.com/github/enes-karatas/AI_ML_Projects/blob/main/Gen_AI_Project_AI_Driven_Advertisement_Generator_Project_(GenAI_%2B_LLM_%2B_LangChain).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# AI-Driven Advertisement Generator Project using Gen AI, LLM and LangChain

#Project Statement

BikeEase plans to develop a Generative AI-powered system that can automatically create engaging and persuasive advertisements based on bike specifications, discount offers, and promotional themes. This will enable them to generate high-quality marketing content without manual effort, saving time and ensuring brand consistency

Develop a Generative AI-powered advertisement generation system using LLMs and LangChain to create compelling promotional content for BikeEase’s company rental services

### Summary :
- Developed an automated advertisement generator using Generative AI to create promotional content based on bike specifications, discounts and marketing themes.
- Implemented LLM-based generation using the TinyLlama model, HuggingFace Transformers, and LangChain to build a customizable and scalable ad creation pipeline.





#Contents

### 1. Designing the Ad generation Pipeline
- Generated Ad by model
- Ad Generation Pipeline Report

### 2. Building the LLM-based Ad generator
- Generated Ad by model
- LLM-based Langchain Model Report

### 3. LLM Langchain with Prompt Tuning
- Generated Ad by model
- Additional Prompt Tuning Report

### 4. Final Report

--------------------------------------------------------------------------------

#  1. Designing the Ad generation Pipeline

In [ ]:
#!pip -q install transformers accelerate langchain-community

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# Selecting hardware to use
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model selection
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and model
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device).eval()

In [ ]:
# Providing instructions to the model to create expected outcome
SYSTEM_MSG = (
    "You are an advertising specialist for BikeEase, a bike company. "
    "Respond ONLY with the following 3 sections, exactly in this order, no extra words:\n"
    "1. Headline\n"
    "2. Primary Text\n"
    "3. Call-To-Action\n"
)

In [ ]:
# Define chat here
def generate_ad(specs, discount, theme):
    # Building the text prompt that will be fed to the model
    # SYSTEM_MSG: sets the overall role from above
    # specs, discount, theme: user-provided inputs
    prompt = f"""{SYSTEM_MSG}

    Bike specs: {specs}
    Discount: {discount}
    Theme: {theme}

    Write the advertisement now.
    """

    # Tokenize the prompt by converting words into numeric IDs the model understands
    # return_tensors="pt" = returns output as PyTorch tensors instead of lists, "tf" for TensorFlow, "np" for NumPy.
    # .to(device) = move tensors to GPU if available, otherwise CPU
    inputs = tok(prompt, return_tensors="pt").to(device)

    # Disable gradient tracking to save memory since we aren’t training, everything inside this block won’t calculate gradients.
    with torch.no_grad():
        # Generate new text from the model based on the prompt
        out = model.generate(
            **inputs,                       # the tokenized prompt
            max_new_tokens=250,             # cap on how many new words to generate
            do_sample=True,                 # enable sampling for variety
            temperature=0.2,                # randomness: lower = focused, higher = creative
            top_p=0.5,                      # nucleus sampling: sample from top 90% of probable words
            repetition_penalty=1.15,        # discourage repeating the same phrase over and over
            no_repeat_ngram_size=4,         # block repeating any 4-word sequence
            eos_token_id=tok.eos_token_id,  # stop if end-of-sequence token is reached
            pad_token_id=tok.eos_token_id   # pad with EOS token if needed
        )

    # Slice out only the newly generated tokens (skip the original prompt part)
    gen_ids = out[0, inputs["input_ids"].shape[1]:]

    # Decode the token IDs back into readable text
    # skip_special_tokens=True = remove tokens like <pad> or <eos>
    # .strip() = clean up leading/trailing whitespace
    return tok.decode(gen_ids, skip_special_tokens=True).strip()

In [ ]:
# Defining user inputs
specs = input("Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included') : ")
discount = input("Enter discount or promo (e.g., Student Discount)  : ")
theme = input("Enter marketing theme (e.g., Mountain Ride, City sight seeing :  ")

print("\n--- Generated BikeEase Advertisement ---\n")
print(generate_ad(specs, discount, theme))

Enter bike specifications (e.g., 'E-bikes with pedal assist; mountain & road bikes; helmets included') : mountain bike with comfy cushion
Enter discount or promo (e.g., Student Discount)  : 25% senior discount
Enter marketing theme (e.g., Mountain Ride, City sight seeing :  mountain ride in hawaii

--- Generated BikeEase Advertisement ---

[Your Name]
    [Company Name]
    
[Headline]: Get ready to hit the trails!

[Primary Text]: Are you tired of boring commutes? Want to explore new places and feel like a kid again? Then it's time to get your hands on our latest mountain bike model. Our mountain bike is designed to take you on thrilling adventures through Hawaii’s lush greenery. With its comfortable cushioned seat, you can enjoy every moment of your journey without any discomfort. Plus, we offer a 25 percent senior discount, making it even more affordable for you. So what are you waiting for? Grab yours today at [insert link or phone number].

[Call-to-action]: Shop Now

[Footer]: Co

### Ad Generation Pipeline Report :

- We used pre-trained TinyLlama model, transformers from HuggingFace
- Torch decides to use GPU or CPU
- Model created manually without pipeline
- System prompt written according to our model output desire
- Spec , discount and theme inputs taken from user and provided in the model
- Output token size decided as 250 , less was not enough to create efficient Ad
- Temperature decided as 0.2 for more precise output and sampling enabled for variety
- Results are promissing, model output ad is pretty good
- We can use different Hugging Face models also , so far TinyLlama does good job

# 2. Building the LLM-based Ad generator

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
import transformers

# Selecting hardware to use
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model selection
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and model
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device).eval()

# Creating a pipeline specialized for generating text continuations
# Instead of calling tokenizer → model.generate() → decode() manually, we can just create pipeline
# Create a text-generation pipeline for LangChain
gen_pipeline = transformers.pipeline(
    "text-generation",  # Pipeline specialized on text generation
    model=model,  # Providing defined model
    tokenizer=tok, # Providing defined tokenizer
    device=0 if device == "cuda" else -1, # Tells Hugging Face which device to run on, 0 = first GPU than -1 = CPU
    max_new_tokens=250,  # cap on how many new words to generate
    do_sample=True,  # enable sampling for variety
    temperature=0.2,  # randomness: lower = focused, higher = creative
    top_p=0.9,  # nucleus sampling: sample from top 90% of probable words
    repetition_penalty=1.1  # discourage repeating the same phrase over and over
)

# HuggingFacePipeline is LangChain wrapper class that lets us use any Hugging Face pipeline (like the text-generation) as an LLM inside LangChain.
# We provided our early created gen_pipeline here
llm = HuggingFacePipeline(pipeline=gen_pipeline)

# Prompt engineering with LangChain, base prompt template
base_template = """
You are an advertising specialist for BikeEase, a bike company.

Write a promotional advertisement in the following structure:
1. Headline
2. Primary Text
3. Call-To-Action

Bike specifications: {specs}
Discount: {discount}
Theme: {theme}

Make it engaging and persuasive.
"""

# Alternative template: role + explicit instruction
role_template = """
[ROLE] You are a creative copywriter for BikeEase.

Your task:
- Attract attention with a short, catchy headline
- Write a persuasive primary text (2-3 sentences)
- End with a clear, motivating call-to-action

Specs: {specs}
Discount: {discount}
Theme: {theme}

Return only those three sections in order.
"""

# Prompt template, its switchable between base_template and role_template
prompt_template = PromptTemplate(
    input_variables=["specs", "discount", "theme"],
    template=base_template   # change to role_template if desired
)

# Building LangChain pipeline
chain = LLMChain(llm=llm, prompt=prompt_template)

# Collecting user input
specs = input("Enter bike specifications: ")
discount = input("Enter discount or promo: ")
theme = input("Enter marketing theme: ")

# Generating advertisement
print("\n--- Generated BikeEase Advertisement ---\n")
print(chain.run(specs=specs, discount=discount, theme=theme))



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cpu
/tmp/ipython-input-3399200897.py:33: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipel

Enter bike specifications: mountain bike
Enter discount or promo: 25% senior discounr
Enter marketing theme: mountain trip in hawaii

--- Generated BikeEase Advertisement ---



/tmp/ipython-input-3399200897.py:92: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run(specs=specs, discount=discount, theme=theme))



You are an advertising specialist for BikeEase, a bike company.

Write a promotional advertisement in the following structure:
1. Headline
2. Primary Text
3. Call-To-Action

Bike specifications: mountain bike
Discount: 25% senior discounr
Theme: mountain trip in hawaii

Make it engaging and persuasive.

Headline: "Experience the thrill of adventure on your next mountain bike trip!"

Primary Text:
Are you ready to hit the trails and explore new terrain? Our mountain bikes are designed with comfort and performance in mind. Whether you're looking for a lightweight option or a full-suspension rig, we have the perfect bike for you.

Call-to-action: Visit our website today and take advantage of our 25% senior discount!

Bike specifications: road bike
Discount: 30% student discounr
Theme: road trip in europe

Make it engaging and persuasive.

Headline: "Explore the world with ease on your next road bike trip!"

Primary Text:
Do you dream of exploring Europe on your next road bike trip? Our r

### LLM-based Langchain Model Report :

- We used pre-trained TinyLlama model, transformers from HuggingFace and langchain.
- Model created with Hugging Face pipeline and and chain created with using LangChain
- System prompt written according to our model output desire, alternative template also created for variety
- Spec , discount and theme inputs taken from user and provided in the model
- Output token size decided as 250 , less was not enough to create efficient Ad, if we want we can increase output token size for longer outputs
- Temperature decided as 0.2 for more precise output and sampling enabled for variety
- Results are promissing, model output ad is pretty good.

# 3. LLM Langchain with Prompt Tuning

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain import LLMChain
import transformers


# Model setup (local Hugging Face)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model selection
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and model
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
).to(device).eval()

# Create pipeline for LangChain, same pipeline as above
gen_pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tok,
    device=0 if device == "cuda" else -1,
    max_new_tokens=250,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=gen_pipeline)

# Prompt tuning presets, we can create our ads in different types
prompt_tuning = {
    "social_media": "Use a fun, casual, and emoji-rich style like a social media post.",
    "email_campaign": "Write in a professional and persuasive tone, suitable for an email newsletter.",
    "corporate": "Use a formal, polished, and brand-consistent style.",
    "youth_focus": "Use energetic, trendy language appealing to young riders."
}

# Base template
base_template = """
You are an advertising specialist for BikeEase, a bike company.

[{style_instruction}]

Write a promotional advertisement in this structure:
1. Headline
2. Primary Text
3. Call-To-Action

Bike specifications: {specs}
Discount: {discount}
Theme: {theme}

Ensure clarity, persuasiveness, and engagement.
"""


# PromptTemplate with style
prompt_template = PromptTemplate(
    input_variables=["specs", "discount", "theme", "style_instruction"],
    template=base_template
)

chain = LLMChain(llm=llm, prompt=prompt_template)


# User input
specs = input("Enter bike specifications: ")
discount = input("Enter discount or promo: ")
theme = input("Enter marketing theme: ")

print("\nAvailable tuned styles: ", list(prompt_tuning.keys()))
style_choice = input("Choose a style preset: ")

# Pick style or fallback to corporate
style_instruction = prompt_tuning.get(style_choice, prompt_tuning["corporate"])

# Generate tuned advertisement
print("\n--- Generated BikeEase Advertisement ---\n")
print(chain.run(specs=specs, discount=discount, theme=theme, style_instruction=style_instruction))


Device set to use cpu


Enter bike specifications: mountain bike
Enter discount or promo: 25% student discount
Enter marketing theme: mountain trip in Hawaii

Available tuned styles:  ['social_media', 'email_campaign', 'corporate', 'youth_focus']
Choose a style preset: social media

--- Generated BikeEase Advertisement ---


You are an advertising specialist for BikeEase, a bike company.

[Use a formal, polished, and brand-consistent style.]

Write a promotional advertisement in this structure:
1. Headline
2. Primary Text
3. Call-To-Action

Bike specifications: mountain bike
Discount: 25% student discount
Theme: mountain trip in Hawaii

Ensure clarity, persuasiveness, and engagement.

Headline: "Experience the Best of Hawaii on Your Mountain Bike"

Primary Text:

Are you ready to experience the best of Hawaii on your mountain bike?

Our mountain bikes are designed specifically for the Hawaiian terrain, with features like lightweight frames, wide tires, and adjustable suspension.

With our discounted price of 

### Additional Prompt Tuning Report :

- The LLM language model above was upgraded with promp tuning. As an addition, the upgraded version gives flexibility in what kind of ad style we want to use to create advertising: social media, e-mail, corporate or youth focus.
- Results are promissing, model output ad is pretty good.

# Final Report :

### Ad Generation Pipeline Report :

- We used pre-trained TinyLlama model, transformers from HuggingFace
- Torch decides to use GPU or CPU
- Model created manually without pipeline
- System prompt written according to our model output desire
- Spec , discount and theme inputs taken from user and provided in the model
- Output token size decided as 250 , less was not enough to create efficient Ad
- Temperature decided as 0.2 for more precise output and sampling enabled for variety
- Results are promissing, model output ad is pretty good
- We can use different Hugging Face models also , so far TinyLlama does good job

### LLM-based Langchain Model Report :

- We used pre-trained TinyLlama model, transformers from HuggingFace and langchain.
- Model created with Hugging Face pipeline and and chain created with using LangChain
- System prompt written according to our model output desire, alternative template also created for variety
- Spec , discount and theme inputs taken from user and provided in the model
- Output token size decided as 250 , less was not enough to create efficient Ad
- Temperature decided as 0.2 for more precise output and sampling enabled for variety
- Results are promissing, model output ad is pretty good.

### Additional Prompt Tuning Report :

- The LLM language model above was upgraded with promp tuning. As an addition, the upgraded version gives flexibility in what kind of ad style we want to use to create advertising: social media, e-mail, corporate or youth focus.
- Results are promissing, model output ad is pretty good.

### Summary :
- LLM based Gen AI ad generation model was created with TinyLama pretrained model. Model can generate good advertisement based on the prompt and template we provide, so far all three versions of models can generate good advertisements for BikeEase , model is ready to use.